<a href="https://colab.research.google.com/github/vishgithubuser/AZ400-DesigningandImplementingMicrosoftDevOpsSolutions/blob/master/Cash_Flow_Simulation_and_Daily_Spending_Limit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
from datetime import date, timedelta
import math
from typing import List, Dict, Any, Tuple

# --- Configuration Constants ---
SIMULATION_DAYS = 60  # How far into the future to simulate (e.g., 60 days)
CC_GRACE_DAYS = 21   # Standard number of days after statement date that payment is due.

# --- Data Structures for Inputs ---

def get_due_date(start_date: date, day_of_month: int, day_offset: int) -> date:
    """Calculates the exact date for a payment based on a day of the month and an offset."""
    # Find the next statement generation date
    if start_date.day < day_of_month:
        statement_date = start_date.replace(day=day_of_month)
    else:
        # Statement already generated this month, calculate for next month
        if start_date.month == 12:
            statement_date = date(start_date.year + 1, 1, day_of_month)
        else:
            statement_date = start_date.replace(month=start_date.month + 1, day=day_of_month)

    # Calculate the due date based on the offset
    due_date = statement_date + timedelta(days=day_offset)
    return due_date

def get_next_pay_date(current_date: date, pay_details: Dict[str, Any]) -> date:
    """Calculates the next pay date based on frequency and day type."""
    frequency = pay_details['frequency'].lower()
    day_spec = pay_details['day_spec']

    if frequency == 'monthly':
        # Day spec is day of the month (e.g., 15)
        if current_date.day < day_spec:
            return current_date.replace(day=day_spec)
        else:
            # Must be next month
            y, m = current_date.year, current_date.month
            if m == 12:
                return date(y + 1, 1, day_spec)
            else:
                return date(y, m + 1, day_spec)

    elif frequency == 'bi-weekly':
        # Day spec is a reference date (e.g., last payday)
        reference_date = datetime.datetime.strptime(day_spec, '%Y-%m-%d').date()
        days_diff = (current_date - reference_date).days

        # Calculate days until next payday (14 days cycle)
        days_until_next = 14 - (days_diff % 14)
        if days_until_next == 14: # If it lands exactly on the reference date
            days_until_next = 0

        # If it's today or a future date in this cycle
        return current_date + timedelta(days=days_until_next)

    return current_date # Should not happen

class CashFlowPlanner:
    """
    Manages the cash flow simulation, scheduling income and expenses,
    and calculating the safe daily spending limit.
    """
    def __init__(self, today: date, credit_cards: List[Dict], paychecks: List[Dict], fixed_expenses: List[Dict]):
        self.today = today
        self.credit_cards = credit_cards
        self.paychecks = paychecks
        self.fixed_expenses = fixed_expenses
        self.simulation_end = self.today + timedelta(days=SIMULATION_DAYS)
        self.cash_balance = 0.0 # Starting balance (can be set by the user if known)
        self.schedule = {} # Date-based schedule of all future transactions

    def _schedule_cc_payments(self):
        """Calculates and schedules the required credit card payments."""
        for cc in self.credit_cards:
            # We assume the current balance needs to be paid on the next due date
            cc_due_date = get_due_date(
                self.today,
                cc['statement_day_of_month'],
                CC_GRACE_DAYS
            )

            if cc_due_date <= self.simulation_end:
                amount = cc['current_balance']

                # Check if a payment for this CC is already scheduled on the due date (from the user input)
                # If not, schedule the calculated balance payment

                # We prioritize paying on the due date, and we only schedule if the amount is > 0
                if amount > 0:
                    description = f"CC Payment: {cc['name']} (Due)"

                    if cc_due_date not in self.schedule:
                        self.schedule[cc_due_date] = []

                    self.schedule[cc_due_date].append({
                        'type': 'Expense',
                        'amount': amount,
                        'description': description
                    })

    def _schedule_fixed_expenses(self):
        """Schedules recurring fixed bills within the simulation window."""
        for expense in self.fixed_expenses:
            current_date = self.today

            # Find the first payment date
            first_pay_date = get_due_date(
                current_date,
                expense['pay_day_of_month'],
                day_offset=0
            )

            # Schedule all recurring payments until the end of the simulation
            while first_pay_date <= self.simulation_end:
                if first_pay_date >= self.today:
                    if first_pay_date not in self.schedule:
                        self.schedule[first_pay_date] = []

                    self.schedule[first_pay_date].append({
                        'type': 'Expense',
                        'amount': expense['amount'],
                        'description': f"Fixed Expense: {expense['name']}"
                    })

                # Move to the next month for the expense
                if first_pay_date.month == 12:
                    first_pay_date = date(first_pay_date.year + 1, 1, expense['pay_day_of_month'])
                else:
                    first_pay_date = first_pay_date.replace(month=first_pay_date.month + 1)

    def _schedule_paychecks(self):
        """Schedules all recurring paychecks within the simulation window."""
        for pay in self.paychecks:
            current_date = self.today
            next_pay_date = get_next_pay_date(current_date, pay)

            while next_pay_date <= self.simulation_end:
                if next_pay_date >= self.today:
                    if next_pay_date not in self.schedule:
                        self.schedule[next_pay_date] = []

                    self.schedule[next_pay_date].append({
                        'type': 'Income',
                        'amount': pay['amount'],
                        'description': f"Income: {pay['name']}"
                    })

                # Move to the next pay cycle (assuming monthly or bi-weekly for simplicity)
                if pay['frequency'].lower() == 'monthly':
                    y, m = next_pay_date.year, next_pay_date.month
                    if m == 12:
                        next_pay_date = date(y + 1, 1, pay['day_spec'])
                    else:
                        next_pay_date = date(y, m + 1, pay['day_spec'])
                elif pay['frequency'].lower() == 'bi-weekly':
                    next_pay_date += timedelta(days=14)
                else:
                    break # Stop if unsupported frequency

    def generate_schedule(self):
        """Generates the full transaction schedule."""
        self.schedule = {}
        self._schedule_cc_payments()
        self._schedule_fixed_expenses()
        self._schedule_paychecks()

        # Sort the schedule by date
        self.schedule = dict(sorted(self.schedule.items()))


    def run_simulation(self, starting_balance: float = 0.0) -> Tuple[float, List[Dict]]:
        """
        Runs the daily simulation and calculates the daily safe spending amount.

        Returns the final calculated safe spending limit and the simulation log.
        """
        self.cash_balance = starting_balance
        self.generate_schedule()

        log = []

        # Calculate total net flow for the simulation period
        total_income = sum(t['amount'] for date, transactions in self.schedule.items()
                           for t in transactions if t['type'] == 'Income')
        total_expense = sum(t['amount'] for date, transactions in self.schedule.items()
                            for t in transactions if t['type'] == 'Expense')

        # Net cash buffer for the entire period
        net_buffer = starting_balance + total_income - total_expense

        # Find the number of days in the period from today (inclusive)
        days_in_period = (self.simulation_end - self.today).days + 1

        # Simple initial calculation: Total net buffer divided by days
        # This is the "safe" daily spend that leaves the buffer at $0 at the end.
        if days_in_period > 0:
            daily_safe_spend = max(0.0, net_buffer / days_in_period)
        else:
            daily_safe_spend = 0.0

        print(f"\n--- Cash Flow Summary ({SIMULATION_DAYS} Days) ---")
        print(f"Starting Balance: ${starting_balance:,.2f}")
        print(f"Total Projected Income: +${total_income:,.2f}")
        print(f"Total Projected Expenses (Fixed + CCs): -${total_expense:,.2f}")
        print(f"Net Final Buffer (if you spend $0/day): ${net_buffer:,.2f}")
        print("--------------------------------------------------\n")

        # --- Daily Simulation and Refined Spend Calculation ---

        current_date = self.today
        while current_date <= self.simulation_end:
            # 1. Apply scheduled transactions for the day
            net_day_flow = 0.0
            daily_events = []

            if current_date in self.schedule:
                for t in self.schedule[current_date]:
                    flow = t['amount'] if t['type'] == 'Income' else -t['amount']
                    self.cash_balance += flow
                    net_day_flow += flow
                    daily_events.append(t['description'])

            # 2. Recalculate the rolling net buffer from this day forward
            # This is the total cash needed to cover all future expenses.

            remaining_expenses = sum(
                t['amount'] for date, transactions in self.schedule.items()
                for t in transactions if t['type'] == 'Expense' and date > current_date
            )

            remaining_income = sum(
                t['amount'] for date, transactions in self.schedule.items()
                for t in transactions if t['type'] == 'Income' and date > current_date
            )

            # Cash available after today's transactions + future income
            future_available_cash = self.cash_balance + remaining_income

            # This is the absolute minimum cash balance we need today, plus $0 buffer.
            required_future_cash = remaining_expenses

            # The remaining funds we have available to spend between now and the end of the simulation
            remaining_spendable_budget = future_available_cash - required_future_cash

            # Days remaining in the window (including today)
            days_remaining = (self.simulation_end - current_date).days + 1

            # Calculate the adjusted daily safe spending limit
            if days_remaining > 0:
                # Distribute the remaining budget evenly across the remaining days
                current_daily_limit = max(0.0, remaining_spendable_budget / days_remaining)
            else:
                current_daily_limit = remaining_spendable_budget # Last day, spend the rest

            # --- Log the day's results ---
            log.append({
                'date': current_date.isoformat(),
                'opening_balance': self.cash_balance - net_day_flow,
                'transactions': daily_events,
                'closing_balance': self.cash_balance,
                'daily_safe_spend_limit': current_daily_limit,
            })

            # Move to the next day
            current_date += timedelta(days=1)

            # Apply the daily limit to the balance for the next day's calculation
            # We assume the user spent the full daily safe limit (this simulates the "worst case" scenario)
            self.cash_balance -= current_daily_limit

        # Return the last calculated daily limit as the recommendation for today
        if log:
            return log[0]['daily_safe_spend_limit'], log
        else:
            return 0.0, []


# --- Example Usage ---

# 1. Define inputs for the simulation
# IMPORTANT: Use a realistic starting date and current date
TODAY = date(2025, 12, 1)

# Current Cash Balance (Amount currently in bank account)
STARTING_CASH = 3000.00

# Credit Cards: (name, current_balance, statement_day_of_month)
CREDIT_CARDS = [
    {
        'name': 'Chase Sapphire',
        'current_balance': 1200.00, # This is the amount due on the next due date
        'statement_day_of_month': 5, # Statement generates on the 5th of the month
    },
    {
        'name': 'Amex Gold',
        'current_balance': 550.00,
        'statement_day_of_month': 20, # Statement generates on the 20th of the month
    },
]

# Paychecks: (name, amount, frequency, day_spec (Day of month or reference date))
PAYCHECKS = [
    {
        'name': 'Primary Job',
        'amount': 2500.00,
        'frequency': 'monthly',
        'day_spec': 15, # Paid on the 15th
    },
    {
        'name': 'Side Gig',
        'amount': 800.00,
        'frequency': 'monthly',
        'day_spec': 30, # Paid on the 30th
    }
]

# Fixed Expenses: (name, amount, pay_day_of_month)
FIXED_EXPENSES = [
    {
        'name': 'Rent',
        'amount': 1800.00,
        'pay_day_of_month': 1, # Due on the 1st
    },
    {
        'name': 'Electricity',
        'amount': 150.00,
        'pay_day_of_month': 10, # Due on the 10th
    },
    {
        'name': 'Internet & Mobile',
        'amount': 100.00,
        'pay_day_of_month': 25, # Due on the 25th
    }
]

# 2. Initialize and Run the Planner
planner = CashFlowPlanner(TODAY, CREDIT_CARDS, PAYCHECKS, FIXED_EXPENSES)
safe_limit, simulation_log = planner.run_simulation(STARTING_CASH)


# 3. Output Results

print("==================================================")
print(f"| TODAY'S DATE: {TODAY.isoformat()} ")
print(f"| SIMULATION PERIOD: {SIMULATION_DAYS} Days (Until {planner.simulation_end.isoformat()})")
print("==================================================")
print(f"| RECOMMENDED DAILY SAFE SPENDING LIMIT: ${safe_limit:,.2f}")
print("==================================================")

print("\n--- Daily Simulation Log (First 15 Days) ---")
print(f"{'Date':<10} | {'Opening Balance':>18} | {'Daily Limit':>13} | {'Closing Balance':>18} | Events")
print("-" * 88)
for i, day in enumerate(simulation_log):
    if i < 15:
        events_str = ", ".join(day['transactions']) if day['transactions'] else "None"

        # Determine color for closing balance
        balance_color = ""
        if day['closing_balance'] < 0:
            balance_color = " (!!! OVERDRAWN !!!)"

        print(f"{day['date']:<10} | ${day['opening_balance']:>17,.2f} | ${day['daily_safe_spend_limit']:>11,.2f} | ${day['closing_balance']:>17,.2f}{balance_color} | {events_str}")

    elif i == 15:
        print("... Log truncated. See full 'simulation_log' for all 60 days.")

print("-" * 88)

# You can inspect the full log for a detailed view of every day
# print(simulation_log)

ValueError: day is out of range for month